In [2]:
import rasterio as rio
import numpy as np

In [5]:
with rio.open("https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cli_024_sea_level_rise/cli_024_sea_level_rise_ft.tif", "r") as src:
    profile = src.profile
    num_bands = profile["count"]
    print(num_bands)
    #data = src.read(indexes=1)
    windows = src.block_windows()
    for win_ix, win in windows:
        new_data = np.zeros((256, 256))
        print(new_data.shape)
        
        # Loop over windows in the CC data, add values in the new_data block
        for bnd in range(num_bands,0, -1):
            print(bnd)
            print(win, win_ix)
            
            hardcoded_window = ((100000, 100200),(200000, 200200))
            data = src.read(window=hardcoded_window, indexes=1)
            print(data)
            break 
        break
    
print(profile)

10
(256, 256)
10
((0, 256), (0, 256)) (0, 0)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 432360, 'height': 140517, 'count': 10, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.000832639467111, 0.0, -180.0,
       0.0, -0.000832639467111, 61.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}


In [34]:
from rasterio.crs import CRS
from affine import Affine

sample_tif = "climate_central_merge_sample.tif"
profile = {
    'driver': 'GTiff', 
   'dtype': 'uint8', 
   'nodata': 0.0, 
   'width': 256, 
   'height': 256, 
   'count': 10, 
   'crs': CRS({'init': 'epsg:4326'}), 
   'transform': Affine(0.000832639467111, 0.0, -180.0,
0.0, -0.000832639467111, 61.0), 
   'blockxsize': 256, 
   'blockysize': 256, 
   'tiled': True, 
   'compress': 'lzw', 
   'interleave': 'pixel'
}

num_bands = profile["count"]

with rio.open(sample_tif, "w", **profile) as dst:
    # Make data that conforms to the CC style... 
    # i.e. if an earlier band has a 1 in a cell, the later ones do too
    
    new_data = np.zeros((256, 256), dtype="uint8")
    
    for bnd in range(1,num_bands+1):
        print("Writing band", bnd)
        new_ones = np.random.choice([True,False], new_data.shape)
        new_data[new_ones] = 1
        dst.write(new_data, indexes=bnd)

Writing band 1
Writing band 2
Writing band 3
Writing band 4
Writing band 5
Writing band 6
Writing band 7
Writing band 8
Writing band 9
Writing band 10


In [36]:
with rio.open(sample_tif, "r") as src:
        profile = src.profile
        num_bands = profile["count"]
        for bnd in range(num_bands, 0, -1):
            print("Copying data from band", bnd)
            data = src.read(indexes=bnd)
            merged_data[data==1] = bnd

Copying data from band 10
Copying data from band 9
Copying data from band 8
Copying data from band 7
Copying data from band 6
Copying data from band 5
Copying data from band 4
Copying data from band 3
Copying data from band 2
Copying data from band 1


In [42]:
sample_merged_tif = "climate_central_merged_sample.tif"
profile = {
    'driver': 'GTiff', 
   'dtype': 'uint8', 
   'nodata': 0.0, 
   'width': 256, 
   'height': 256, 
   'count': 1, 
   'crs': CRS({'init': 'epsg:4326'}), 
   'transform': Affine(0.000832639467111, 0.0, -180.0,
0.0, -0.000832639467111, 61.0), 
   'blockxsize': 256, 
   'blockysize': 256, 
   'tiled': True, 
   'compress': 'lzw', 
   'interleave': 'pixel'
}

# Open the file to contain the merged data
with rio.open(sample_merged_tif, "w", **profile) as dst:
    # Iterate over the input data, updating the merged file as we go
    # Would do this for every block in the larger block_windows
    
    merged_data = np.zeros((256,256), dtype="uint8")
    
    with rio.open(sample_tif, "r") as src:
        profile = src.profile
        num_bands = profile["count"]
        for bnd in range(num_bands, 0, -1):
            print("Copying data from band", bnd)
            data = src.read(indexes=bnd)
            merged_data[data==1] = bnd
    
    print(merged_data.shape)
    print(merged_data)
    dst.write(merged_data, indexes=1)

Copying data from band 10
Copying data from band 9
Copying data from band 8
Copying data from band 7
Copying data from band 6
Copying data from band 5
Copying data from band 4
Copying data from band 3
Copying data from band 2
Copying data from band 1
(256, 256)
[[5 1 1 ..., 5 4 3]
 [1 1 5 ..., 4 5 1]
 [1 1 4 ..., 3 5 1]
 ..., 
 [1 1 5 ..., 3 3 1]
 [9 1 3 ..., 2 2 1]
 [2 6 3 ..., 3 1 1]]


In [43]:
with rio.open(sample_merged_tif, "r", **profile) as src:
    merged_profile = src.profile
    print(merged_profile)
    data = src.read(indexes=1)
    
print(data)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 256, 'height': 256, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.000832639467111, 0.0, -180.0,
       0.0, -0.000832639467111, 61.0), 'blockxsize': '256', 'blockysize': '256', 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
[[5 1 1 ..., 5 4 3]
 [1 1 5 ..., 4 5 1]
 [1 1 4 ..., 3 5 1]
 ..., 
 [1 1 5 ..., 3 3 1]
 [9 1 3 ..., 2 2 1]
 [2 6 3 ..., 3 1 1]]
